In [1]:
%cd D:\Project\Toolkit_for_Preprocessing_MXH\viHSD_tokenize

from IPython.display import clear_output
clear_output()

In [2]:
import tensorflow as tf
import torch

tf.keras.backend.clear_session()
# clear gpu memory using torch
torch.cuda.empty_cache()
# clear output
clear_output()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
train_path = (r"Data\train.csv")
dev_path = (r"Data\dev.csv")
test_path = (r"Data\test.csv")
model_name = 'vinai/phobert-base'
test_index = 50 # default None value
batch_size = 4
max_len = 128
lr = 5e-6
epochs = 2
shuffle = False

In [4]:
from Code.Dataset import split_path, create_dataloader

if test_index != None and test_index > 3:
    # Load the data
    train_path, dev_path, test_path = split_path(test_path, test_index, train_path, dev_path, test_path)
elif test_index != None: 
    print("Test index out of range. Please provide a valid interger index greater than 3.")


Training set: 35 samples
Test set: 7 samples
Development set: 8 samples


In [5]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer, AutoModel
)


classes = ['0', '1', '2']
tokenizer = AutoTokenizer.from_pretrained(model_name)
embedding_model = AutoModel.from_pretrained(model_name)
classification_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(classes)  # Number of classes
)
clear_output()

# Adjust the token embeddings size if needed
embedding_model.resize_token_embeddings(len(tokenizer))


Embedding(64001, 768, padding_idx=1)

In [6]:
train_dataloader = create_dataloader(train_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len, shuffle=False)
dev_dataloader = create_dataloader(dev_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len, shuffle=False)
test_dataloader = create_dataloader(test_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len, shuffle=False)

# Get the first batch of data from the DataLoader
first_batch = next(iter(test_dataloader))

# Access input_ids, attention_mask, and labels
input_ids = first_batch['input_ids']
attention_mask = first_batch['attention_mask']
labels = first_batch['label']

# Print to check
print(f"Input IDs: {input_ids.size()}")
print(f"Attention Mask: {attention_mask.size()}")
print(f"Labels: {labels.shape}")

Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Input IDs: torch.Size([4, 128])
Attention Mask: torch.Size([4, 128])
Labels: torch.Size([4, 7])


In [7]:
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np
import torch.optim as optim


In [8]:
# Set up the model
from Code.Model import setup_model, MultiTaskModel, train, test

model, criterion, optimizer, device, num_epochs = setup_model(
    model_class = MultiTaskModel, 
    embedding_model = embedding_model, 
    classification_model = classification_model,
    lr=5e-6,
    weight_decay=1e-5,
    num_epochs=2
    )

In [9]:
# Assuming you've set up the model, dataloaders, criterion, and optimizer
train(model, train_dataloader, dev_dataloader, criterion, optimizer, device, num_epochs)

Epoch: 1


Training Loss: 1.9249
Validation Loss: 1.9834
Macro F1-Score: 0.4615
Epoch: 2


Training Loss: 1.8486
Validation Loss: 1.9621
Macro F1-Score: 0.4615


In [10]:

predictions, true_labels = test(model, test_dataloader, device)

Testing: 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]

Macro F1 Score: 0.4615


In [1]:
%cd D:\Project\Toolkit_for_Preprocessing_MXH\VSMEC_tokenize

from IPython.display import clear_output
clear_output()

In [2]:
!python main.py --model "vinai/phobert-base" --train_path "Data\train.csv" --dev_path "Data\dev.csv" --test_path "Data\test.csv" --batch_size 8 --max_len 128 --lr 5e-6 --num_epochs 2 --output_json "test_results.json" --output_dir "output" --test_index 50

Using device: cuda
Training set: 35 samples
Test set: 7 samples
Development set: 8 samples
Columns: Index(['index', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Text', 'Tag', 'sentence_id'], dtype='object')
Input IDs: torch.Size([6, 128])
Attention Mask: torch.Size([6, 128])
Labels: torch.Size([6, 3])
Epoch: 1
Training Loss: 1.0536
Validation Loss: 1.0308
Macro F1-Score: 0.4000
Epoch: 2
Training Loss: 0.9967
Validation Loss: 0.9750
Macro F1-Score: 0.4000
Macro F1 Score: 0.4000
Test results saved to test_results.json
Model saved to output\trained_model.pth


C:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bi